In [ ]:
%matplotlib inline
#%pylab
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as mpc
import matplotlib.dates as dts
import numpy as np
import pandas as pd
import itertools
import os
#import ROOT
import datetime
#from root_numpy import root2array, root2rec, tree2rec, array2root
from scipy.optimize import curve_fit
#from scipy.misc import factorial


In [ ]:
plt.rcParams.update({'font.size': 16})

In [ ]:
from decode_new import GetWaveformsNoiseRemoval
from decode_new import GetWaveforms

In [ ]:
from datetime import datetime

In [ ]:
def gettime(name):
    date = name.split('PrM1-')[-1].split('.dat')[0]
    time = datetime.strptime(date,'%Y_%m_%d-%H_%M_%S')
    return time

In [ ]:
def QaQc(wf_v):
    anode_v = wf_v[0][700:]
    cathode_v = wf_v[1][500:]
    Qa = np.amax(anode_v)
    Qc = np.amin(cathode_v)
    return Qa,Qc

def QaQcSTATS(ev_wf_v):
    
    Qa_v = []
    Qc_v = []
    for wf_v in ev_wf_v:
        Qa,Qc = QaQc(wf_v)
        Qa_v.append(Qa*1e3)
        Qc_v.append(Qc*1e3)
    Qa_v = np.array(Qa_v)
    Qc_v = np.array(Qc_v)

    Qa_avg = np.mean(Qa_v)
    Qa_rms = np.std(Qa_v)
    Qc_avg = np.mean(Qc_v)
    Qc_rms = np.std(Qc_v)
    
    return [[Qa_avg,Qa_rms],[Qc_avg,Qc_rms]]

# Set path to files

In [ ]:
#PATH = '/Users/alexanderantonakis/Desktop/Software/LArCADe/data/Run4/' # Liquid
PATH = '/Users/alexanderantonakis/Desktop/Software/LArCADe/data/Run5/' # Gas
#PHASE = "Liquid"
PHASE = "Gas"

subdir_v = os.listdir(PATH)
dt = 0.0005

In [ ]:
VOLTAGEMAP = {}

datafile_v = []


# Voltages for which we have multiple pressures
special_gas_voltages = []

for subdir in subdir_v:
    #print(subdir)
    fulldir = PATH + subdir + '/'
    voltage = 0
    #print(subdir.split('VA')[-1].split('G'))
    if subdir.split('VA')[-1].split('G')[-1] == '2':
        #print("Skip G2 for gas")
        voltage = int(subdir.split('VA')[-1].split('G')[0])
        special_gas_voltages.append(voltage)
        #continue
        
    try:
        voltage = int(subdir.split('VA')[-1])
    except:
        #v1 = subdir.split('VA')[-1]
        voltage = int(subdir.split('VA')[-1].split('G')[0])
        
    #print(voltage)
    datafiles = os.listdir(fulldir)
    for datafile in datafiles:
        #print(datafile)
        if (datafile.find('.dat') >= 0):
            VOLTAGEMAP[fulldir+datafile] = voltage
            datafile_v.append(fulldir+datafile)

In [ ]:
def getDataFile(time):
    found = False
    for i,datafile in enumerate(datafile_v):
        if (datafile.find(time) >= 0):
            return i
    return None

# List all available files

In [ ]:
#for i,datafile in enumerate(datafile_v):
    #print ('%i -> %s'%(i,datafile))
"""
new_data_files = []
if PHASE == 'Gas':
    for i,datafile in enumerate(datafile_v):
        l = datafile.split('/')[8]
        if l[-1] != '2':
            new_data_files.append(datafile)
            #print(l)
            
datafile_v = new_data_files   
"""

# Example plotting average waveform

In [ ]:
"""
time = '12_10_01'
time = str(gettime(datafile_v[0]))
print(time)
filepathname = ''
nfile = getDataFile(time)
if (nfile != None):
    datafile = datafile_v[nfile]
    filepathname = datafile
    print (datafile)
    #ev_wf_v = GetWaveformsNoiseRemoval(datafile,verbose=True)
    ev_wf_v = GetWaveforms(datafile)
else:
    print ('no input file found!')
    
#print (len(ev_wf_v))
#print(ev_wf_v)
"""

In [ ]:
#time_v = np.linspace(0,dt*len(ev_wf_v[0][0]),len(ev_wf_v[0][0]))

In [ ]:
"""
avg_A_wf_v = np.zeros(len(time_v))
avg_C_wf_v = np.zeros(len(time_v))
for n in range(len(ev_wf_v)):
    avg_A_wf_v += ev_wf_v[n][0]
    avg_C_wf_v += ev_wf_v[n][1]
avg_A_wf_v /= len(ev_wf_v)
avg_C_wf_v /= len(ev_wf_v)
"""

In [ ]:
"""
fig = plt.figure(figsize=(10,6))
plt.plot(time_v,avg_A_wf_v,color='r',label='Anode',lw=3)
plt.plot(time_v,avg_C_wf_v,color='b',label='Cathode',lw=3)
plt.grid()
plt.legend(fontsize=20)
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Amplitude [V]',fontsize=20)
plt.title('Run in Liquid @ %i V'%VOLTAGEMAP[filepathname],fontsize=20)
plt.ylim([-0.07,0.07])
#plt.xlim([0.1,1.3])

plt.show()
"""

In [ ]:
def integrate(time_series, ampl_series, is_anode):
    if is_anode:
        tlow = 0.2
        thigh = 1.8
        index_low = 0
        index_high = 0
        for num in range(len(time_series)):
            if time_series[num] > tlow:
                index_low = num -1
                break
        for num in range(len(time_series)):
            if time_series[num] > thigh:
                index_high = num -1
                break
                
        sum_tot = 0
        for num in ampl_series[index_low:index_high]:
            if num > 0:
                sum_tot += num
        return sum_tot
        #return sum(ampl_series[index_low:index_high])
        
    else:
        tlow = 0.0
        thigh = 1.8
        index_low = 0
        index_high = 0
        for num in range(len(time_series)):
            if time_series[num] > tlow:
                index_low = num -1
                break
        for num in range(len(time_series)):
            if time_series[num] > thigh:
                index_high = num -1
                break
        
        sum_tot = 0
        for num in ampl_series[index_low:index_high]:
            if num < 0:
                sum_tot += num
        return -1*sum_tot
        #return -1*sum(ampl_series[index_low:index_high])
        
        

In [ ]:
from scipy import signal
import scipy.interpolate as interpolate


x = np.array([ 0. ,  1.2,  1.9,  3.2,  4. ,  6.5])
y = np.array([ 0. ,  2.3,  3. ,  4.3,  2.9,  3.1])

t, c, k = interpolate.splrep(x, y, s=0, k=4)
print('''\
t: {}
c: {}
k: {}
'''.format(t, c, k))
N = 100
xmin, xmax = x.min(), x.max()
xx = np.linspace(xmin, xmax, N)
spline = interpolate.BSpline(t, c, k, extrapolate=False)

plt.plot(x, y, 'bo', label='Original points')
plt.plot(xx, spline(xx), 'r', label='BSpline')
plt.grid()
plt.legend(loc='best')
plt.show()


def get_spline(time_series, ampl_series, window_length=71):
    y_smooth = signal.savgol_filter(ampl_series, window_length=71, polyorder=3, mode="nearest")
    t, c, k = interpolate.splrep(time_series, y_smooth, s=0, k=4)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    return spline
    

# Waveform by waveform Plotting

In [ ]:
# time-step
dt = 0.0005

# where to compute baseline
BaselineRange1 = [0.10,0.15]
BaselineRange2 = [0.18,0.20]

# where to integrate charge
IntegrationRange = [0.2, 0.6]

B1Tmin = int(BaselineRange1[0]/dt)
B1Tmax = int(BaselineRange1[1]/dt)

B2Tmin = int(BaselineRange2[0]/dt)
B2Tmax = int(BaselineRange2[1]/dt)

IntMin = int(IntegrationRange[0]/dt)
IntMax = int(IntegrationRange[1]/dt)

# below, one entry per Run
# average waveforms for anode and cathode
ALL_AVG_C_WF_V = []
ALL_AVG_A_WF_V = []
# max/min amplitude at anode and cathode
AVG_A_MAX_V = []
AVG_C_MIN_V = []
# voltage
VOLTAGE_V = []

INTEGRAL_A_V = []
INTEGRAL_C_V = []


time_arrays = []
voltages = []
anode_arrays = []
cathode_arrays = []


bad_file = '/Users/alexanderantonakis/Desktop/Software/LArCADe/data/Run5/VA4000G/PrM1-2024_08_22-15_00_58.dat'
#fig = plt.figure(figsize=(8,6))

#for date,voltage in VOLTAGEMAP.iteritems():
for date,voltage in VOLTAGEMAP.items():
    
    if voltage in special_gas_voltages:
        print("Special voltage", voltage)
        nfile = getDataFile(date)
        datafile = 0
        if (nfile != None):
            datafile = datafile_v[nfile]
        print("Special Datafile", datafile)
        print(datafile.split('VA')[-1].split('G')[-1])
        if datafile.split('VA')[-1].split('G')[-1].split('/')[0] != '2':
            print("Skip double voltages in gas")
            print(datafile.split('VA')[-1])
            continue
    
    
    
    #VOLTAGE_V.append(voltage)
    
    nfile = getDataFile(date)
    if (nfile != None):
        #datafile = PATH + datafile_v[nfile]
        datafile = datafile_v[nfile]
        if datafile == bad_file:
            continue
        print(datafile)
        #ev_wf_v = GetWaveformsNoiseRemoval(datafile)
        ev_wf_v = GetWaveforms(datafile)
    else:
        print('no input file found!')
        continue
    
    VOLTAGE_V.append(voltage)
    time_v = np.linspace(0,dt*len(ev_wf_v[0][0]),len(ev_wf_v[0][0]))
    #time_v = np.linspace(0,dt*len(ev_wf_v[0]),len(ev_wf_v[0]))
    
    avg_A_wf_v = np.zeros(len(time_v))
    avg_C_wf_v = np.zeros(len(time_v))
    #for n in xrange(len(ev_wf_v)):
    #    avg_A_wf_v += ev_wf_v[n][0]
    #    avg_C_wf_v += ev_wf_v[n][1]
    #avg_A_wf_v /= len(ev_wf_v)
    #avg_C_wf_v /= len(ev_wf_v)
    
    
    ANODE_MAX_V = []
    CATHODE_MAX_V = []
    
    # used waveforms
    NWFS = 0
    
    for n in range(len(ev_wf_v)):
        #print(ev_wf_v[n][1])
        cathode_wf = 1e3 * np.array(ev_wf_v[n][1])
        anode_wf   = 1e3 * np.array(ev_wf_v[n][0])
        
        # compute baselines
        B1C = np.mean(cathode_wf[B1Tmin:B1Tmax])
        B2C = np.mean(cathode_wf[B2Tmin:B2Tmax])
        B1A = np.mean(anode_wf[B1Tmin:B1Tmax])
        B2A = np.mean(anode_wf[B2Tmin:B2Tmax])
        #print (r'baseline C1 : %.02f C2 : %.02f A1 : %.02f A2 : %.02f [$\mu$ V]'%(B1C,B2C,B1A,B2A))
        
        BC = 0.5*(B1C+B2C)
        BA = 0.5*(B1A+B2A)
        
        #cathode_wf -= BC
        #anode_wf   -= BA
        
        AMAX = np.max(anode_wf)
        ANODE_MAX_V.append(AMAX)
        
        CMAX = np.max(cathode_wf)
        CATHODE_MAX_V.append(CMAX)
        
        # skip noise
        #if (AMAX > 50): continue
        
        # save avg wf
        avg_A_wf_v += anode_wf
        avg_C_wf_v += cathode_wf
        NWFS += 1
        
        '''
        fig = plt.figure(figsize=(10,6))
        plt.plot(time_v,anode_wf,color='r',label='Anode',lw=1,alpha=0.5)
        plt.plot(time_v,cathode_wf,color='b',label='Cathode',lw=1,alpha=0.5)
        plt.grid()
        #plt.legend(fontsize=20)
        plt.xlabel('Time [ms]',fontsize=20)
        plt.ylabel('Amplitude [mV]',fontsize=20)
        plt.title('Run in Liquid @ %i V'%voltage,fontsize=20)
        plt.xlim([0,0.75])
        plt.ylim([-100,100])
        plt.show()
        '''
        
        #if (n > 50):
        #    break
        
    if NWFS == 0:
        NWFS = 1
        
    avg_A_wf_v /= NWFS
    avg_C_wf_v /= NWFS
    
    ALL_AVG_C_WF_V.append(avg_C_wf_v)
    ALL_AVG_A_WF_V.append(avg_A_wf_v)
    #TIME_V.append
    
    AVG_A_MAX_V.append(np.max(avg_A_wf_v))
    AVG_C_MIN_V.append(np.min(avg_C_wf_v))
    
    #integ_A = integrate(time_v, avg_A_wf_v, 1)
    #integ_C = integrate(time_v, avg_C_wf_v, 0)
    
    curr_spline_A = get_spline(time_v, avg_A_wf_v)
    curr_spline_C = get_spline(time_v, avg_C_wf_v)
    
    integ_A = integrate(time_v, curr_spline_A(time_v), 1)
    integ_C = integrate(time_v, curr_spline_C(time_v), 0)
    
    time_arrays.append(time_v)
    voltages.append(voltage)
    anode_arrays.append(avg_A_wf_v)
    cathode_arrays.append(avg_C_wf_v)
    
    
    INTEGRAL_A_V.append(integ_A)
    INTEGRAL_C_V.append(integ_C)
    
    fig = plt.figure(figsize=(10,6))
    plt.plot(time_v,avg_A_wf_v,color='r',label='Anode',lw=3,alpha=1)
    plt.plot(time_v,avg_C_wf_v,color='b',label='Cathode',lw=3,alpha=1)
    plt.grid()
    plt.legend(fontsize=20)
    plt.xlabel('Time [ms]',fontsize=20)
    plt.ylabel('Amplitude [mV]',fontsize=20)
    plt.title('Run in '+PHASE + ' @ %i V'%voltage,fontsize=20)
    #plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
    plt.xlim([0,1.8])
    plt.ylim([1.1*min(avg_C_wf_v), 1.1*max(avg_A_wf_v)])
    plt.show()
    
    fig = plt.figure(figsize=(10,6))
    plt.plot(time_v,curr_spline_A(time_v),color='r',label='Anode Spline',lw=3,alpha=1)
    plt.plot(time_v,curr_spline_C(time_v),color='b',label='Cathode Spline',lw=3,alpha=1)
    plt.grid()
    plt.legend(fontsize=20)
    plt.xlabel('Time [ms]',fontsize=20)
    plt.ylabel('Amplitude [mV]',fontsize=20)
    plt.title('Run in '+PHASE + ' @ %i V'%voltage,fontsize=20)
    #plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
    plt.xlim([0,1.8])
    plt.ylim([1.1*min(avg_C_wf_v), 1.1*max(avg_A_wf_v)])
    plt.show()
    

        
#break

In [ ]:
COLORMAP = {}
COLORMAP[6000] = 'k'
COLORMAP[5000] = 'b'
COLORMAP[4000] = 'r'
COLORMAP[3000] = 'orange'
COLORMAP[2000] = 'c'
COLORMAP[1000] = 'gray'
COLORMAP[900] = 'yellow'
COLORMAP[800] = 'green'
COLORMAP[700] = 'coral'
COLORMAP[600] = 'olive'
COLORMAP[500] = 'salmon'

In [ ]:

FIG_PATH = '/Users/alexanderantonakis/Desktop/Software/LArCADe/'

AMPL_RATIO_V = -np.array(AVG_A_MAX_V)/np.array(AVG_C_MIN_V)

INTEGRAL_RATIO_V = np.array(INTEGRAL_A_V)/np.array(INTEGRAL_C_V)

new_v = []
new_ratio = []

curr_A_sum = AVG_A_MAX_V[0]
curr_C_sum = AVG_C_MIN_V[0]
curr_n = 1
prev_voltage = VOLTAGE_V[0]
for num in range(1, len(AVG_C_MIN_V)):
    if VOLTAGE_V[num] == prev_voltage:
        curr_A_sum += AVG_A_MAX_V[num]
        curr_C_sum += AVG_C_MIN_V[num]
        curr_n += 1
    else:
        new_v.append(prev_voltage)
        new_ratio.append(-1*curr_A_sum / curr_C_sum)
        curr_A_sum = AVG_A_MAX_V[num]
        curr_C_sum = AVG_C_MIN_V[num]
        curr_n = 1
        prev_voltage = VOLTAGE_V[num]
        
fig = plt.figure(figsize=(8,6))
plt.title('Run in '+PHASE,fontsize=20)
plt.plot(VOLTAGE_V,AMPL_RATIO_V,'ro')
plt.xlabel('anode voltage [V]')
plt.ylabel('Charge Amplitude Ratio')
plt.xlim([0,8000])
plt.ylim([0,4])
plt.plot([0, 8000], [1, 1], c='b', linestyle="--")
plt.plot([5300, 5300], [0, 4], c='b', linestyle="--")
plt.savefig(FIG_PATH+'ampl_ratio_'+PHASE+'.png',dpi=250)
plt.show()

fig = plt.figure(figsize=(8,6))
plt.title('Run in '+PHASE,fontsize=20)
plt.plot(new_v,new_ratio,'ro')
plt.xlabel('anode voltage [V]')
plt.ylabel('Average Charge Amplitude Ratio')
plt.xlim([0,8000])
plt.ylim([0.0, 2])
plt.plot([0, 8000], [1, 1], c='b', linestyle="--")
plt.plot([5300, 5300], [0, 2], c='b', linestyle="--")
plt.savefig(FIG_PATH+'avg_ampl_ratio_'+PHASE+'.png',dpi=250)
plt.show()


fig = plt.figure(figsize=(8,6))
plt.title('Run in '+PHASE,fontsize=20)
plt.plot(VOLTAGE_V, INTEGRAL_RATIO_V,'ro')
plt.xlabel('anode voltage [V]')
plt.ylabel('Integrated Charge Ratio [A/C]')
plt.xlim([0,8000])
plt.ylim([0.0, 2])
plt.plot([0, 8000], [1, 1], c='b', linestyle="--")
plt.plot([5300, 5300], [0, 2], c='b', linestyle="--")
plt.savefig(FIG_PATH+'integral_ratio_'+PHASE+'.png',dpi=250)
plt.show()



avg_integral_v = []
avg_integral_ratio = []
unique_v = list(set(VOLTAGE_V))
for v in unique_v:
    avg_integral_v.append(v)
    ratios = []
    for num in range(len(VOLTAGE_V)):
        if VOLTAGE_V[num] == v:
            ratios.append(INTEGRAL_RATIO_V[num])

    m = np.mean(np.array(ratios))
    avg_integral_ratio.append(m)

fig = plt.figure(figsize=(8,6))
plt.title('Run in '+PHASE,fontsize=20)
plt.plot(avg_integral_v, avg_integral_ratio,'ro')
plt.xlabel('Anode Voltage [V]', fontsize=16)
plt.ylabel('Average Integrated Charge Ratio [A/C]')
plt.xlim([0,5200])
plt.ylim([0,2])
#plt.ylim([0.0, 1.1*max(avg_integral_ratio)])
plt.plot([0, 8000], [1, 1], c='b', linestyle="--", linewidth=3)
plt.plot([5300, 5300], [0, 1.1*max(avg_integral_ratio)], c='b', linestyle="--")
plt.savefig(FIG_PATH+'avg_integral_ratio_'+PHASE+'.png',dpi=250)
plt.show()

# Anode Waveforms

In [ ]:
# Let's plot waveforms of the same voltages together

avg_time_series_arr = []
avg_anode_wvfs = []

unique_voltages = list(set(voltages))
for v in unique_voltages:
    indices = np.where(np.array(voltages) == v)[0]
    fig = plt.figure(figsize=(10,6))
    
    avg_wvf = np.zeros_like(time_arrays[indices[0]])
    maxes = []
    mins = []
    N = 0
    for i in indices:
        if max(anode_arrays[i]) > 200:
            continue
        maxes.append(max(anode_arrays[i]))
        mins.append(min(anode_arrays[i]))
        plt.plot(time_arrays[i], anode_arrays[i],lw=3,alpha=1)
        avg_wvf += anode_arrays[i]
        N += 1
        
    avg_wvf /= N
    avg_anode_wvfs.append(avg_wvf)
    avg_time_series_arr.append(time_arrays[indices[0]])
    plt.plot(time_arrays[indices[0]], avg_wvf, lw=3,alpha=1, label="avg")
    
    plt.grid()
    plt.legend(fontsize=20)
    plt.xlabel('Time [ms]',fontsize=20)
    plt.ylabel('Amplitude [mV]',fontsize=20)
    plt.title('Run in '+PHASE + ' @ %i V'%v,fontsize=20)
    #plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
    #plt.xlim([0,1.8])
    plt.ylim([1.1*min(mins), 1.1*max(maxes)])
    plt.show()

In [ ]:

avg_cathode_wvfs = []

for v in unique_voltages:
    indices = np.where(np.array(voltages) == v)[0]
    fig = plt.figure(figsize=(10,6))
    
    avg_wvf = np.zeros_like(time_arrays[indices[0]])
    maxes = []
    mins = []
    N = 0
    for i in indices:
        if max(cathode_arrays[i]) > 200:
            continue
        maxes.append(max(cathode_arrays[i]))
        mins.append(min(cathode_arrays[i]))
        plt.plot(time_arrays[i], cathode_arrays[i],lw=3,alpha=1)
        avg_wvf += cathode_arrays[i]
        N += 1
        
    avg_wvf /= N
    avg_cathode_wvfs.append(avg_wvf)
    plt.plot(time_arrays[indices[0]], avg_wvf, lw=3,alpha=1, label="avg")
    
    plt.grid()
    plt.legend(fontsize=20)
    plt.xlabel('Time [ms]',fontsize=20)
    plt.ylabel('Amplitude [mV]',fontsize=20)
    plt.title('Run in '+PHASE + ' @ %i V'%v,fontsize=20)
    #plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
    #plt.xlim([0,1.8])
    plt.ylim([1.1*min(mins), 1.1*max(maxes)])
    plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
mins = []
maxes = []
for num in range(len(unique_voltages)):    
    
    plt.plot(avg_time_series_arr[num], avg_anode_wvfs[num], lw=3,alpha=1, label=str(unique_voltages[num])+" V")
    mins.append(min(avg_anode_wvfs[num]))
    maxes.append(max(avg_anode_wvfs[num]))

plt.grid()
#plt.legend()
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Average Amplitude [mV]',fontsize=20)
plt.title('Run in '+PHASE+" : Anode",fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([1,1.1*max(avg_time_series_arr[0])]) # late time
plt.xlim([min(avg_time_series_arr[0]), 0.75]) # early time
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
plt.yscale('log')
#plt.ylim([1.1*min(mins), 1.1*max(maxes)]) # full y range
plt.ylim([1, 1.1*max(maxes)])
#plt.ylim([1, 30])
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
mins = []
maxes = []
for num in range(len(unique_voltages)):    
    
    plt.plot(avg_time_series_arr[num], avg_anode_wvfs[num], lw=3,alpha=1, label=str(unique_voltages[num])+" V")
    mins.append(min(avg_anode_wvfs[num]))
    maxes.append(max(avg_anode_wvfs[num]))
    
plt.grid()
#plt.legend()
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Average Amplitude [mV]',fontsize=20)
plt.title('Run in '+PHASE+" : Anode",fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([1,1.1*max(avg_time_series_arr[0])]) # late time
plt.xlim([min(avg_time_series_arr[0]), 0.75]) # early time
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
plt.yscale('log')
plt.xscale('log')
plt.xlim([0.4, 0.75]) # early time
#plt.ylim([1.1*min(mins), 1.1*max(maxes)]) # full y range
plt.ylim([1, 1.1*max(maxes)])
#plt.ylim([1, 30])
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,6))
mins = []
maxes = []

anode_integrals = []
for num in range(len(unique_voltages)):    
    curr_spline_A = get_spline(avg_time_series_arr[num], avg_anode_wvfs[num], 101)
    plt.plot(avg_time_series_arr[num], curr_spline_A(avg_time_series_arr[num]), lw=3,alpha=1, label=str(unique_voltages[num])+" V")
    mins.append(min(avg_anode_wvfs[num]))
    maxes.append(max(avg_anode_wvfs[num]))
    integ_A = integrate(avg_time_series_arr[num], curr_spline_A(avg_time_series_arr[num]), 1)
    anode_integrals.append(integ_A)

plt.grid()
#plt.legend()
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Average Amplitude [mV]',fontsize=20)
plt.title('Run in '+PHASE+" : Anode Splines",fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([0,1.8])
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
#plt.yscale('log')
plt.ylim([1.1*min(mins), 1.1*max(maxes)])
plt.show()

In [ ]:
plt.scatter(unique_voltages, anode_integrals)
plt.show()

In [ ]:
anode_integrals = []
cathode_integrals = []

fig = plt.figure(figsize=(8,6))
mins = []
maxes = []
for num in range(len(unique_voltages)):    
    
    plt.plot(avg_time_series_arr[num], avg_cathode_wvfs[num], lw=3,alpha=1, label=str(unique_voltages[num])+" V")
    mins.append(min(avg_cathode_wvfs[num]))
    maxes.append(max(avg_cathode_wvfs[num]))
    mask1 = (avg_cathode_wvfs[num] <= 0)
    mask2 = (avg_time_series_arr[num] < 1.5)
    mask = mask1 & mask2
    cathode_integrals.append(-1*sum(avg_cathode_wvfs[num][mask]))

    # anode masks
    mask3 = (avg_anode_wvfs[num] >= 0)
    mask4 = (avg_time_series_arr[num] < 1.5)
    mask5 = (avg_time_series_arr[num] > 0.25)
    mask = mask3 & mask4 & mask5
    anode_integrals.append(sum(avg_anode_wvfs[num][mask]))   
    
plt.grid()
#plt.legend()
plt.xlabel('Time [ms]',fontsize=20)
plt.ylabel('Average Amplitude [mV]',fontsize=20)
plt.title('Run in '+PHASE+" : Cathode",fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([1,1.1*max(avg_time_series_arr[0])]) # late time
#plt.xlim([min(avg_time_series_arr[0]), 0.75]) # early time
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
#plt.yscale('log')
plt.xlim([0.75, 2.25])
#plt.ylim([1.1*min(mins), 1.1*max(maxes)]) # full y range
#plt.ylim([1, 1.1*max(maxes)])
plt.ylim([-5, 2.5])
plt.show()


fig = plt.figure(figsize=(8,6))
plt.grid()
#plt.legend()
plt.scatter(unique_voltages, cathode_integrals)
plt.xlabel('Anode Voltage',fontsize=20)
plt.ylabel('Charge Integral of Average Waveform',fontsize=20)
plt.title('Run in '+PHASE+" : Cathode",fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([1,1.1*max(avg_time_series_arr[0])]) # late time
#plt.xlim([min(avg_time_series_arr[0]), 0.75]) # early time
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
#plt.yscale('log')
plt.xscale('log')
#plt.ylim([1.1*min(mins), 1.1*max(maxes)]) # full y range
#plt.ylim([1, 1.1*max(maxes)])
#plt.ylim([1, 30])
plt.show()



fig = plt.figure(figsize=(8,6))
plt.grid()
#plt.legend()
plt.scatter(unique_voltages, cathode_integrals, label="Cathode")
plt.scatter(unique_voltages, anode_integrals, label="Anode")
plt.xlabel('Anode Voltage',fontsize=20)
plt.ylabel('Charge Integral of Average Waveform',fontsize=20)
plt.title('Run in '+PHASE,fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([1,1.1*max(avg_time_series_arr[0])]) # late time
#plt.xlim([min(avg_time_series_arr[0]), 0.75]) # early time
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
#plt.yscale('log')
plt.legend(fontsize=20)
plt.xscale('log')
#plt.ylim([1.1*min(mins), 1.1*max(maxes)]) # full y range
#plt.ylim([1, 1.1*max(maxes)])
#plt.ylim([1, 30])
plt.show()




integral_ratio = np.array(anode_integrals) / np.array(cathode_integrals)

fig = plt.figure(figsize=(8,6))
plt.grid()
#plt.legend()
plt.scatter(unique_voltages, integral_ratio, label="Cathode")
plt.xlabel('Anode Voltage',fontsize=20)
plt.ylabel('Charge Integral Ratio of Averaged Waveforms [A/C]',fontsize=20)
plt.title('Run in '+PHASE,fontsize=20)
#plt.savefig('/home/david/Neutrinos/LArCADe/Analysis/2021/avg_%iv.pdf'%voltage,dpi=250)
#plt.xlim([1,1.1*max(avg_time_series_arr[0])]) # late time
#plt.xlim([min(avg_time_series_arr[0]), 0.75]) # early time
#plt.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
#plt.yscale('log')
plt.legend(fontsize=20)
plt.xscale('log')
plt.plot([min(unique_voltages), max(unique_voltages)], [1, 1], c='r', linestyle="--")
plt.ylim([0.9*min(integral_ratio), 1.1]) # full y range
#plt.ylim([1, 1.1*max(maxes)])
#plt.ylim([1, 30])
plt.show()



plt.hist2d(cathode_integrals, anode_integrals, bins=30)
plt.show()

# Timing results